# E-Commerce Backend API Comprehensive Testing

This notebook provides thorough testing of all existing functions and routes in the e-commerce backend before moving to Phase 6. We'll test:

- ✅ Server Health & Infrastructure
- 🔐 Authentication System (Registration, Login, Profile)
- 📂 Category Management (Public & Admin routes)
- 📦 Product Management (Public & Admin routes)
- 🛒 Shopping Cart Functionality
- 📝 Order Management System
- ⚠️ Error Handling & Edge Cases
- 🔒 Security & Authorization

## Test Status Dashboard
- **Total Tests**: 0
- **Passed**: 0  
- **Failed**: 0
- **Coverage**: All major endpoints and business logic

In [ ]:
# Import Required Libraries
import requests
import pandas as pd
import json
import time
import random
import string
from datetime import datetime
from typing import Dict, List, Tuple, Optional
import matplotlib.pyplot as plt
import networkx as nx
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# API Configuration
BASE_URL = 'http://localhost:3000'
API_BASE = f'{BASE_URL}/api/v1'

# Test Results Tracker
class TestTracker:
    def __init__(self):
        self.results = []
        self.passed = 0
        self.failed = 0
        
    def add_result(self, test_name: str, status: str, details: str = "", response_data: dict = None):
        result = {
            'test_name': test_name,
            'status': status,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'details': details,
            'response_data': response_data
        }
        self.results.append(result)
        if status == 'PASS':
            self.passed += 1
        else:
            self.failed += 1
    
    def get_summary(self):
        total = len(self.results)
        return {
            'total': total,
            'passed': self.passed,
            'failed': self.failed,
            'success_rate': f"{(self.passed/total*100):.1f}%" if total > 0 else "0%"
        }
    
    def print_summary(self):
        summary = self.get_summary()
        print(f"📊 Test Summary:")
        print(f"   Total Tests: {summary['total']}")
        print(f"   ✅ Passed: {summary['passed']}")
        print(f"   ❌ Failed: {summary['failed']}")
        print(f"   📈 Success Rate: {summary['success_rate']}")

# Initialize tracker
tracker = TestTracker()

# Helper functions
def generate_unique_email():
    timestamp = str(int(time.time()))
    random_suffix = ''.join(random.choices(string.ascii_lowercase, k=5))
    return f"test_{timestamp}_{random_suffix}@example.com"

def make_request(method: str, endpoint: str, headers: Dict = None, data: Dict = None, auth_token: str = None):
    """Make HTTP request with proper error handling"""
    url = f"{API_BASE}{endpoint}"
    request_headers = {'Content-Type': 'application/json'}
    
    if headers:
        request_headers.update(headers)
    
    if auth_token:
        request_headers['Authorization'] = f'Bearer {auth_token}'
    
    try:
        if method.upper() == 'GET':
            response = requests.get(url, headers=request_headers)
        elif method.upper() == 'POST':
            response = requests.post(url, headers=request_headers, json=data)
        elif method.upper() == 'PUT':
            response = requests.put(url, headers=request_headers, json=data)
        elif method.upper() == 'DELETE':
            response = requests.delete(url, headers=request_headers)
        else:
            raise ValueError(f"Unsupported HTTP method: {method}")
        
        return response
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

print("✅ Libraries imported and testing framework initialized!")

## 1. 📊 Server Health & Infrastructure Testing

First, we'll verify that the server is running and responding correctly to basic requests.

In [ ]:
# Test 1: Server Health Check
print("🔍 Testing Server Health...")

try:
    response = requests.get(f"{BASE_URL}/health")
    
    if response.status_code == 200:
        data = response.json()
        if data.get('success') == True:
            tracker.add_result("Server Health Check", "PASS", f"Server is healthy - Status: {response.status_code}", data)
            print("✅ Server Health Check: PASSED")
            print(f"   ├─ Status: {response.status_code}")
            print(f"   ├─ Message: {data.get('message', 'N/A')}")
            print(f"   ├─ Environment: {data.get('environment', 'N/A')}")
            print(f"   └─ Timestamp: {data.get('timestamp', 'N/A')}")
        else:
            tracker.add_result("Server Health Check", "FAIL", "Health endpoint returned success=false", data)
            print("❌ Server Health Check: FAILED - Health endpoint returned success=false")
    else:
        tracker.add_result("Server Health Check", "FAIL", f"Unexpected status code: {response.status_code}")
        print(f"❌ Server Health Check: FAILED - Status Code: {response.status_code}")
        
except requests.exceptions.ConnectionError:
    tracker.add_result("Server Health Check", "FAIL", "Server is not running or not accessible")
    print("❌ Server Health Check: FAILED - Server is not running or not accessible")
except Exception as e:
    tracker.add_result("Server Health Check", "FAIL", f"Unexpected error: {str(e)}")
    print(f"❌ Server Health Check: FAILED - Error: {str(e)}")

print()

In [ ]:
# Test 2: Basic API Endpoints Availability
print("🔍 Testing Basic API Endpoints...")

# Test root endpoint
try:
    response = requests.get(BASE_URL)
    if response.status_code == 200:
        data = response.json()
        tracker.add_result("Root Endpoint", "PASS", f"Root endpoint accessible - Status: {response.status_code}", data)
        print("✅ Root Endpoint: PASSED")
        print(f"   ├─ Message: {data.get('message', 'N/A')}")
        print(f"   └─ Version: {data.get('version', 'N/A')}")
    else:
        tracker.add_result("Root Endpoint", "FAIL", f"Unexpected status code: {response.status_code}")
        print(f"❌ Root Endpoint: FAILED - Status Code: {response.status_code}")
except Exception as e:
    tracker.add_result("Root Endpoint", "FAIL", f"Error accessing root: {str(e)}")
    print(f"❌ Root Endpoint: FAILED - Error: {str(e)}")

# Test undefined endpoint (should return 404)
try:
    response = requests.get(f"{API_BASE}/nonexistent-endpoint")
    if response.status_code == 404:
        tracker.add_result("404 Error Handling", "PASS", "Undefined endpoints properly return 404")
        print("✅ 404 Error Handling: PASSED")
    else:
        tracker.add_result("404 Error Handling", "FAIL", f"Expected 404, got {response.status_code}")
        print(f"❌ 404 Error Handling: FAILED - Expected 404, got {response.status_code}")
except Exception as e:
    tracker.add_result("404 Error Handling", "FAIL", f"Error testing 404: {str(e)}")
    print(f"❌ 404 Error Handling: FAILED - Error: {str(e)}")

print()

## 2. 📂 Category Management Testing

Testing all category-related endpoints including public access and data validation.

In [ ]:
# Test 3: Categories Endpoint
print("🔍 Testing Categories Endpoint...")

try:
    response = make_request('GET', '/categories')
    
    if response and response.status_code == 200:
        data = response.json()
        
        if data.get('success') == True:
            categories = data.get('data', [])
            count = data.get('count', 0)
            
            tracker.add_result("Get Categories", "PASS", f"Retrieved {count} categories successfully", data)
            print("✅ Get Categories: PASSED")
            print(f"   ├─ Status: {response.status_code}")
            print(f"   ├─ Success: {data.get('success')}")
            print(f"   ├─ Count: {count}")
            print(f"   └─ Message: {data.get('message', 'N/A')}")
            
            # Store categories for later use
            global categories_data
            categories_data = categories
            
        else:
            tracker.add_result("Get Categories", "FAIL", "Categories endpoint returned success=false", data)
            print("❌ Get Categories: FAILED - success=false")
    else:
        status_code = response.status_code if response else "No Response"
        tracker.add_result("Get Categories", "FAIL", f"Failed with status: {status_code}")
        print(f"❌ Get Categories: FAILED - Status: {status_code}")
        
except Exception as e:
    tracker.add_result("Get Categories", "FAIL", f"Error: {str(e)}")
    print(f"❌ Get Categories: FAILED - Error: {str(e)}")

print()

In [ ]:
# Display Categories in Tabular Format
if 'categories_data' in globals() and categories_data:
    print("📋 Categories Data Analysis:")
    
    # Convert to DataFrame for better visualization
    df_categories = pd.DataFrame(categories_data)
    
    # Select key fields for display
    display_columns = ['name', 'slug', 'description', 'level', 'isActive', 'isFeatured', 'productCount']
    available_columns = [col for col in display_columns if col in df_categories.columns]
    
    if available_columns:
        df_display = df_categories[available_columns].copy()
        
        print(f"\n📊 Categories Summary ({len(df_display)} total):")
        display(df_display)
        
        # Additional analysis
        print(f"\n🔍 Category Analysis:")
        print(f"   ├─ Total Categories: {len(df_categories)}")
        print(f"   ├─ Active Categories: {len(df_categories[df_categories['isActive'] == True])}")
        print(f"   ├─ Featured Categories: {len(df_categories[df_categories['isFeatured'] == True])}")
        print(f"   ├─ Parent Categories (Level 0): {len(df_categories[df_categories['level'] == 0])}")
        print(f"   └─ Child Categories (Level > 0): {len(df_categories[df_categories['level'] > 0])}")
    else:
        print("❌ No displayable columns found in categories data")
else:
    print("❌ No categories data available for display")

print()

In [ ]:
# Test 4: Category Hierarchy & Individual Category Endpoints
print("🔍 Testing Category Hierarchy & Individual Endpoints...")

# Test category hierarchy endpoint
try:
    response = make_request('GET', '/categories/hierarchy')
    
    if response and response.status_code == 200:
        data = response.json()
        tracker.add_result("Category Hierarchy", "PASS", f"Status: {response.status_code}", data)
        print("✅ Category Hierarchy: PASSED")
    else:
        status_code = response.status_code if response else "No Response"
        tracker.add_result("Category Hierarchy", "FAIL", f"Status: {status_code}")
        print(f"❌ Category Hierarchy: FAILED - Status: {status_code}")
        
except Exception as e:
    tracker.add_result("Category Hierarchy", "FAIL", f"Error: {str(e)}")
    print(f"❌ Category Hierarchy: FAILED - Error: {str(e)}")

# Test individual category retrieval
if 'categories_data' in globals() and categories_data and len(categories_data) > 0:
    try:
        # Test with first category
        first_category = categories_data[0]
        category_id = first_category.get('id') or first_category.get('_id')
        
        response = make_request('GET', f'/categories/{category_id}')
        
        if response and response.status_code == 200:
            data = response.json()
            tracker.add_result("Individual Category", "PASS", f"Retrieved category {category_id}", data)
            print(f"✅ Individual Category: PASSED - Retrieved category '{first_category.get('name')}'")
        else:
            status_code = response.status_code if response else "No Response"
            tracker.add_result("Individual Category", "FAIL", f"Status: {status_code}")
            print(f"❌ Individual Category: FAILED - Status: {status_code}")
            
    except Exception as e:
        tracker.add_result("Individual Category", "FAIL", f"Error: {str(e)}")
        print(f"❌ Individual Category: FAILED - Error: {str(e)}")
else:
    tracker.add_result("Individual Category", "SKIP", "No categories data available")
    print("⏭️ Individual Category: SKIPPED - No categories data")

print()

## 3. 📦 Product Management Testing

Testing product endpoints including listing, search, and individual product retrieval.

In [ ]:
# Test 5: Products Endpoint
print("🔍 Testing Products Endpoint...")

try:
    response = make_request('GET', '/products')
    
    if response and response.status_code == 200:
        data = response.json()
        
        if data.get('success') == True:
            products = data.get('data', [])
            count = data.get('count', 0)
            pagination = data.get('pagination', {})
            
            tracker.add_result("Get Products", "PASS", f"Retrieved {count} products successfully", data)
            print("✅ Get Products: PASSED")
            print(f"   ├─ Status: {response.status_code}")
            print(f"   ├─ Success: {data.get('success')}")
            print(f"   ├─ Count: {count}")
            print(f"   ├─ Pagination: {pagination}")
            print(f"   └─ Message: {data.get('message', 'N/A')}")
            
            # Store products for later use
            global products_data
            products_data = products
            
        else:
            tracker.add_result("Get Products", "FAIL", "Products endpoint returned success=false", data)
            print("❌ Get Products: FAILED - success=false")
    else:
        status_code = response.status_code if response else "No Response"
        tracker.add_result("Get Products", "FAIL", f"Failed with status: {status_code}")
        print(f"❌ Get Products: FAILED - Status: {status_code}")
        
except Exception as e:
    tracker.add_result("Get Products", "FAIL", f"Error: {str(e)}")
    print(f"❌ Get Products: FAILED - Error: {str(e)}")

print()

In [ ]:
# Display Products in Tabular Format and Analysis
if 'products_data' in globals() and products_data:
    print("📋 Products Data Analysis:")
    
    # Convert to DataFrame for better visualization
    df_products = pd.DataFrame(products_data)
    
    # Select key fields for display
    display_columns = ['name', 'slug', 'brand', 'status', 'visibility', 'isFeatured', 'inStock', 'isLowStock']
    available_columns = [col for col in display_columns if col in df_products.columns]
    
    if available_columns:
        df_display = df_products[available_columns].copy()
        
        # Add pricing information if available
        if 'pricing' in df_products.columns:
            df_display['price'] = df_products['pricing'].apply(lambda x: x.get('price', 'N/A') if isinstance(x, dict) else 'N/A')
        
        print(f"\n📊 Products Summary ({len(df_display)} total):")
        display(df_display)
        
        # Additional analysis
        print(f"\n🔍 Product Analysis:")
        print(f"   ├─ Total Products: {len(df_products)}")
        
        if 'status' in df_products.columns:
            active_products = len(df_products[df_products['status'] == 'active'])
            print(f"   ├─ Active Products: {active_products}")
        
        if 'isFeatured' in df_products.columns:
            featured_products = len(df_products[df_products['isFeatured'] == True])
            print(f"   ├─ Featured Products: {featured_products}")
        
        if 'inStock' in df_products.columns:
            in_stock_products = len(df_products[df_products['inStock'] == True])
            print(f"   ├─ In Stock Products: {in_stock_products}")
        
        if 'brand' in df_products.columns:
            unique_brands = df_products['brand'].nunique()
            print(f"   └─ Unique Brands: {unique_brands}")
    else:
        print("❌ No displayable columns found in products data")
else:
    print("❌ No products data available for display")

print()

## 4. 🔐 Authentication System Testing

Testing user registration, login, token validation, and protected endpoints.

In [ ]:
# Test 6: User Registration
print("🔍 Testing User Registration...")

# Generate unique test user data
test_email = generate_unique_email()
test_user_data = {
    "firstName": "Test",
    "lastName": "User",
    "email": test_email,
    "password": "Password123!",
    "confirmPassword": "Password123!"
}

try:
    response = make_request('POST', '/auth/register', data=test_user_data)
    
    if response:
        if response.status_code == 201:
            data = response.json()
            if data.get('success') == True:
                tracker.add_result("User Registration", "PASS", f"User registered successfully with email: {test_email}", data)
                print("✅ User Registration: PASSED")
                print(f"   ├─ Status: {response.status_code}")
                print(f"   ├─ Email: {test_email}")
                print(f"   ├─ Success: {data.get('success')}")
                print(f"   └─ User ID: {data.get('data', {}).get('user', {}).get('id', 'N/A')}")
                
                # Store user data for later tests
                global test_user_email, test_user_password
                test_user_email = test_email
                test_user_password = test_user_data['password']
                
            else:
                tracker.add_result("User Registration", "FAIL", "Registration returned success=false", data)
                print("❌ User Registration: FAILED - success=false")
        elif response.status_code == 400:
            data = response.json()
            tracker.add_result("User Registration", "FAIL", f"Validation error: {data.get('message', 'Unknown')}", data)
            print(f"❌ User Registration: FAILED - Validation Error: {data.get('message', 'Unknown')}")
        elif response.status_code == 429:
            tracker.add_result("User Registration", "FAIL", "Rate limit exceeded", response.json() if response.content else {})
            print("❌ User Registration: FAILED - Rate limit exceeded")
        else:
            data = response.json() if response.content else {}
            tracker.add_result("User Registration", "FAIL", f"Unexpected status: {response.status_code}", data)
            print(f"❌ User Registration: FAILED - Status: {response.status_code}")
    else:
        tracker.add_result("User Registration", "FAIL", "No response received")
        print("❌ User Registration: FAILED - No response")
        
except Exception as e:
    tracker.add_result("User Registration", "FAIL", f"Error: {str(e)}")
    print(f"❌ User Registration: FAILED - Error: {str(e)}")

print()

In [ ]:
# Test 7: User Login
print("🔍 Testing User Login...")

if 'test_user_email' in globals() and 'test_user_password' in globals():
    login_data = {
        "email": test_user_email,
        "password": test_user_password
    }
    
    try:
        response = make_request('POST', '/auth/login', data=login_data)
        
        if response:
            if response.status_code == 200:
                data = response.json()
                if data.get('success') == True and data.get('data', {}).get('token'):
                    token = data['data']['token']
                    user_data = data.get('data', {}).get('user', {})
                    
                    tracker.add_result("User Login", "PASS", f"Login successful for {test_user_email}", data)
                    print("✅ User Login: PASSED")
                    print(f"   ├─ Status: {response.status_code}")
                    print(f"   ├─ Email: {test_user_email}")
                    print(f"   ├─ Token: {token[:20]}...")
                    print(f"   └─ User ID: {user_data.get('id', 'N/A')}")
                    
                    # Store auth token for protected endpoint tests
                    global auth_token
                    auth_token = token
                    
                else:
                    tracker.add_result("User Login", "FAIL", "Login response missing token or success=false", data)
                    print("❌ User Login: FAILED - Missing token or success=false")
            elif response.status_code == 401:
                data = response.json() if response.content else {}
                tracker.add_result("User Login", "FAIL", "Invalid credentials", data)
                print("❌ User Login: FAILED - Invalid credentials")
            elif response.status_code == 429:
                tracker.add_result("User Login", "FAIL", "Rate limit exceeded", response.json() if response.content else {})
                print("❌ User Login: FAILED - Rate limit exceeded")
            else:
                data = response.json() if response.content else {}
                tracker.add_result("User Login", "FAIL", f"Unexpected status: {response.status_code}", data)
                print(f"❌ User Login: FAILED - Status: {response.status_code}")
        else:
            tracker.add_result("User Login", "FAIL", "No response received")
            print("❌ User Login: FAILED - No response")
            
    except Exception as e:
        tracker.add_result("User Login", "FAIL", f"Error: {str(e)}")
        print(f"❌ User Login: FAILED - Error: {str(e)}")
else:
    tracker.add_result("User Login", "SKIP", "No registered user available for login test")
    print("⏭️ User Login: SKIPPED - No registered user data")

print()

In [ ]:
# Test 8: Protected Endpoints - User Profile
print("🔍 Testing Protected Endpoints...")

if 'auth_token' in globals():
    try:
        response = make_request('GET', '/auth/me', auth_token=auth_token)
        
        if response:
            if response.status_code == 200:
                data = response.json()
                if data.get('success') == True:
                    user_data = data.get('data', {}).get('user', {})
                    
                    tracker.add_result("User Profile Access", "PASS", "Successfully accessed user profile", data)
                    print("✅ User Profile Access: PASSED")
                    print(f"   ├─ Status: {response.status_code}")
                    print(f"   ├─ User ID: {user_data.get('id', 'N/A')}")
                    print(f"   ├─ Email: {user_data.get('email', 'N/A')}")
                    print(f"   └─ Name: {user_data.get('firstName', '')} {user_data.get('lastName', '')}")
                    
                else:
                    tracker.add_result("User Profile Access", "FAIL", "Profile response success=false", data)
                    print("❌ User Profile Access: FAILED - success=false")
            elif response.status_code == 401:
                data = response.json() if response.content else {}
                tracker.add_result("User Profile Access", "FAIL", "Unauthorized - token might be invalid", data)
                print("❌ User Profile Access: FAILED - Unauthorized (token invalid?)")
            else:
                data = response.json() if response.content else {}
                tracker.add_result("User Profile Access", "FAIL", f"Unexpected status: {response.status_code}", data)
                print(f"❌ User Profile Access: FAILED - Status: {response.status_code}")
        else:
            tracker.add_result("User Profile Access", "FAIL", "No response received")
            print("❌ User Profile Access: FAILED - No response")
            
    except Exception as e:
        tracker.add_result("User Profile Access", "FAIL", f"Error: {str(e)}")
        print(f"❌ User Profile Access: FAILED - Error: {str(e)}")
else:
    tracker.add_result("User Profile Access", "SKIP", "No auth token available")
    print("⏭️ User Profile Access: SKIPPED - No auth token")

# Test access without authentication (should fail)
try:
    response = make_request('GET', '/auth/me')  # No auth token
    
    if response and response.status_code == 401:
        tracker.add_result("Unauthorized Access Protection", "PASS", "Correctly rejected unauthorized access")
        print("✅ Unauthorized Access Protection: PASSED")
    else:
        status_code = response.status_code if response else "No Response"
        tracker.add_result("Unauthorized Access Protection", "FAIL", f"Expected 401, got {status_code}")
        print(f"❌ Unauthorized Access Protection: FAILED - Expected 401, got {status_code}")
        
except Exception as e:
    tracker.add_result("Unauthorized Access Protection", "FAIL", f"Error: {str(e)}")
    print(f"❌ Unauthorized Access Protection: FAILED - Error: {str(e)}")

print()

## 5. 🛒 Shopping Cart Testing

Testing cart operations including adding items, retrieving cart, updating quantities, and removing items.

In [ ]:
# Test 9: Get Cart (Initially Empty)
print("🔍 Testing Cart Functionality...")

# Test getting cart (should be empty initially)
try:
    auth_header = auth_token if 'auth_token' in globals() else None
    response = make_request('GET', '/cart', auth_token=auth_header)
    
    if response:
        if response.status_code == 200:
            data = response.json()
            if data.get('success') == True:
                cart_data = data.get('data', {})
                
                tracker.add_result("Get Cart", "PASS", "Successfully retrieved cart", data)
                print("✅ Get Cart: PASSED")
                print(f"   ├─ Status: {response.status_code}")
                print(f"   ├─ Cart ID: {cart_data.get('_id', 'N/A')}")
                print(f"   ├─ Items Count: {len(cart_data.get('items', []))}")
                print(f"   └─ Total: {cart_data.get('total', 0)}")
                
            else:
                tracker.add_result("Get Cart", "FAIL", "Cart response success=false", data)
                print("❌ Get Cart: FAILED - success=false")
        else:
            data = response.json() if response.content else {}
            tracker.add_result("Get Cart", "FAIL", f"Unexpected status: {response.status_code}", data)
            print(f"❌ Get Cart: FAILED - Status: {response.status_code}")
    else:
        tracker.add_result("Get Cart", "FAIL", "No response received")
        print("❌ Get Cart: FAILED - No response")
        
except Exception as e:
    tracker.add_result("Get Cart", "FAIL", f"Error: {str(e)}")
    print(f"❌ Get Cart: FAILED - Error: {str(e)}")

print()

In [ ]:
# Test 10: Add Item to Cart
print("🔍 Testing Add Item to Cart...")

if 'products_data' in globals() and products_data and len(products_data) > 0:
    # Use first product for testing
    test_product = products_data[0]
    product_id = test_product.get('id') or test_product.get('_id')
    
    add_to_cart_data = {
        "productId": product_id,
        "quantity": 2
    }
    
    try:
        auth_header = auth_token if 'auth_token' in globals() else None
        response = make_request('POST', '/cart/items', data=add_to_cart_data, auth_token=auth_header)
        
        if response:
            if response.status_code in [200, 201]:
                data = response.json()
                if data.get('success') == True:
                    tracker.add_result("Add to Cart", "PASS", f"Added product {product_id} to cart", data)
                    print("✅ Add to Cart: PASSED")
                    print(f"   ├─ Status: {response.status_code}")
                    print(f"   ├─ Product: {test_product.get('name', 'N/A')}")
                    print(f"   ├─ Quantity: {add_to_cart_data['quantity']}")
                    print(f"   └─ Message: {data.get('message', 'N/A')}")
                    
                    # Store cart item info for later tests
                    cart_data = data.get('data', {})
                    if 'items' in cart_data and cart_data['items']:
                        global test_cart_item_id
                        test_cart_item_id = cart_data['items'][-1].get('_id')
                    
                else:
                    tracker.add_result("Add to Cart", "FAIL", "Add to cart response success=false", data)
                    print("❌ Add to Cart: FAILED - success=false")
            elif response.status_code == 400:
                data = response.json() if response.content else {}
                tracker.add_result("Add to Cart", "FAIL", f"Validation error: {data.get('message', 'Unknown')}", data)
                print(f"❌ Add to Cart: FAILED - Validation Error: {data.get('message', 'Unknown')}")
            elif response.status_code == 401:
                tracker.add_result("Add to Cart", "FAIL", "Unauthorized", response.json() if response.content else {})
                print("❌ Add to Cart: FAILED - Unauthorized")
            else:
                data = response.json() if response.content else {}
                tracker.add_result("Add to Cart", "FAIL", f"Unexpected status: {response.status_code}", data)
                print(f"❌ Add to Cart: FAILED - Status: {response.status_code}")
        else:
            tracker.add_result("Add to Cart", "FAIL", "No response received")
            print("❌ Add to Cart: FAILED - No response")
            
    except Exception as e:
        tracker.add_result("Add to Cart", "FAIL", f"Error: {str(e)}")
        print(f"❌ Add to Cart: FAILED - Error: {str(e)}")
else:
    tracker.add_result("Add to Cart", "SKIP", "No products available for cart test")
    print("⏭️ Add to Cart: SKIPPED - No products data")

print()

## 6. 📝 Order Management Testing

Testing order creation, retrieval, and management functionality.

In [ ]:
# Test 11: Create Order (Checkout)
print("🔍 Testing Order Creation...")

if 'auth_token' in globals():
    order_data = {
        "shippingAddress": {
            "street": "123 Test Street",
            "city": "Test City",
            "state": "TS",
            "postalCode": "12345",
            "country": "Test Country"
        },
        "billingAddress": {
            "street": "123 Test Street",
            "city": "Test City", 
            "state": "TS",
            "postalCode": "12345",
            "country": "Test Country"
        },
        "paymentMethod": "credit_card"
    }
    
    try:
        response = make_request('POST', '/orders', data=order_data, auth_token=auth_token)
        
        if response:
            if response.status_code in [200, 201]:
                data = response.json()
                if data.get('success') == True:
                    order = data.get('data', {})
                    
                    tracker.add_result("Create Order", "PASS", f"Order created successfully", data)
                    print("✅ Create Order: PASSED")
                    print(f"   ├─ Status: {response.status_code}")
                    print(f"   ├─ Order ID: {order.get('_id', 'N/A')}")
                    print(f"   ├─ Status: {order.get('status', 'N/A')}")
                    print(f"   ├─ Total: {order.get('total', 'N/A')}")
                    print(f"   └─ Payment Method: {order.get('paymentMethod', 'N/A')}")
                    
                    # Store order ID for later tests
                    global test_order_id
                    test_order_id = order.get('_id') or order.get('id')
                    
                else:
                    tracker.add_result("Create Order", "FAIL", "Order creation response success=false", data)
                    print("❌ Create Order: FAILED - success=false")
            elif response.status_code == 400:
                data = response.json() if response.content else {}
                tracker.add_result("Create Order", "FAIL", f"Validation error: {data.get('message', 'Unknown')}", data)
                print(f"❌ Create Order: FAILED - Validation Error: {data.get('message', 'Unknown')}")
            elif response.status_code == 401:
                tracker.add_result("Create Order", "FAIL", "Unauthorized", response.json() if response.content else {})
                print("❌ Create Order: FAILED - Unauthorized")
            else:
                data = response.json() if response.content else {}
                tracker.add_result("Create Order", "FAIL", f"Unexpected status: {response.status_code}", data)
                print(f"❌ Create Order: FAILED - Status: {response.status_code}")
        else:
            tracker.add_result("Create Order", "FAIL", "No response received")
            print("❌ Create Order: FAILED - No response")
            
    except Exception as e:
        tracker.add_result("Create Order", "FAIL", f"Error: {str(e)}")
        print(f"❌ Create Order: FAILED - Error: {str(e)}")
else:
    tracker.add_result("Create Order", "SKIP", "No auth token available")
    print("⏭️ Create Order: SKIPPED - No auth token")

print()

In [ ]:
# Test 12: Get User Orders
print("🔍 Testing Order Retrieval...")

if 'auth_token' in globals():
    try:
        response = make_request('GET', '/orders', auth_token=auth_token)
        
        if response:
            if response.status_code == 200:
                data = response.json()
                if data.get('success') == True:
                    orders = data.get('data', [])
                    
                    tracker.add_result("Get Orders", "PASS", f"Retrieved {len(orders)} orders", data)
                    print("✅ Get Orders: PASSED")
                    print(f"   ├─ Status: {response.status_code}")
                    print(f"   ├─ Orders Count: {len(orders)}")
                    
                    if orders:
                        latest_order = orders[0]
                        print(f"   ├─ Latest Order ID: {latest_order.get('_id', 'N/A')}")
                        print(f"   └─ Latest Order Status: {latest_order.get('status', 'N/A')}")
                    else:
                        print(f"   └─ No orders found")
                    
                else:
                    tracker.add_result("Get Orders", "FAIL", "Orders response success=false", data)
                    print("❌ Get Orders: FAILED - success=false")
            elif response.status_code == 401:
                tracker.add_result("Get Orders", "FAIL", "Unauthorized", response.json() if response.content else {})
                print("❌ Get Orders: FAILED - Unauthorized")
            else:
                data = response.json() if response.content else {}
                tracker.add_result("Get Orders", "FAIL", f"Unexpected status: {response.status_code}", data)
                print(f"❌ Get Orders: FAILED - Status: {response.status_code}")
        else:
            tracker.add_result("Get Orders", "FAIL", "No response received")
            print("❌ Get Orders: FAILED - No response")
            
    except Exception as e:
        tracker.add_result("Get Orders", "FAIL", f"Error: {str(e)}")
        print(f"❌ Get Orders: FAILED - Error: {str(e)}")
else:
    tracker.add_result("Get Orders", "SKIP", "No auth token available")
    print("⏭️ Get Orders: SKIPPED - No auth token")

# Test individual order retrieval if we have an order ID
if 'test_order_id' in globals() and 'auth_token' in globals():
    try:
        response = make_request('GET', f'/orders/{test_order_id}', auth_token=auth_token)
        
        if response:
            if response.status_code == 200:
                data = response.json()
                if data.get('success') == True:
                    order = data.get('data', {})
                    
                    tracker.add_result("Get Individual Order", "PASS", f"Retrieved order {test_order_id}", data)
                    print("✅ Get Individual Order: PASSED")
                    print(f"   ├─ Order ID: {order.get('_id', 'N/A')}")
                    print(f"   └─ Status: {order.get('status', 'N/A')}")
                    
                else:
                    tracker.add_result("Get Individual Order", "FAIL", "Order response success=false", data)
                    print("❌ Get Individual Order: FAILED - success=false")
            else:
                data = response.json() if response.content else {}
                tracker.add_result("Get Individual Order", "FAIL", f"Status: {response.status_code}", data)
                print(f"❌ Get Individual Order: FAILED - Status: {response.status_code}")
        else:
            tracker.add_result("Get Individual Order", "FAIL", "No response received")
            print("❌ Get Individual Order: FAILED - No response")
            
    except Exception as e:
        tracker.add_result("Get Individual Order", "FAIL", f"Error: {str(e)}")
        print(f"❌ Get Individual Order: FAILED - Error: {str(e)}")

print()

## 7. 📊 Test Results Summary & Analysis

Final test results and recommendations for Phase 6 readiness.

In [ ]:
# Final Test Results Summary
print("=" * 60)
print("🎯 COMPREHENSIVE API TESTING COMPLETE")
print("=" * 60)

# Print summary
tracker.print_summary()

# Detailed results table
if tracker.results:
    print(f"\n📋 Detailed Test Results:")
    df_results = pd.DataFrame(tracker.results)
    
    # Select columns for display
    display_cols = ['test_name', 'status', 'timestamp', 'details']
    available_cols = [col for col in display_cols if col in df_results.columns]
    
    if available_cols:
        df_display = df_results[available_cols].copy()
        
        # Truncate details for better display
        if 'details' in df_display.columns:
            df_display['details'] = df_display['details'].apply(
                lambda x: x[:100] + "..." if len(str(x)) > 100 else x
            )
        
        display(df_display)
    
    # Analysis by status
    status_counts = df_results['status'].value_counts()
    print(f"\n📈 Test Status Breakdown:")
    for status, count in status_counts.items():
        print(f"   {status}: {count}")
    
    # Failed tests details
    failed_tests = df_results[df_results['status'] == 'FAIL']
    if not failed_tests.empty:
        print(f"\n❌ Failed Tests Details:")
        for _, test in failed_tests.iterrows():
            print(f"   • {test['test_name']}: {test['details']}")
    
    # Create simple visualization
    if len(status_counts) > 0:
        plt.figure(figsize=(8, 6))
        colors = ['green' if status == 'PASS' else 'red' if status == 'FAIL' else 'orange' for status in status_counts.index]
        
        plt.pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%', colors=colors)
        plt.title('API Test Results Distribution')
        plt.axis('equal')
        plt.show()

else:
    print("❌ No test results available")

# Phase 6 Readiness Assessment
print(f"\n🎯 PHASE 6 READINESS ASSESSMENT:")
print("-" * 40)

summary = tracker.get_summary()
success_rate = float(summary['success_rate'].replace('%', ''))

if success_rate >= 90:
    print("✅ READY FOR PHASE 6")
    print("   All critical functionality is working correctly.")
elif success_rate >= 75:
    print("⚠️  MOSTLY READY FOR PHASE 6")
    print("   Some minor issues need attention, but core functionality works.")
elif success_rate >= 50:
    print("🔧 NEEDS WORK BEFORE PHASE 6")
    print("   Several issues need to be resolved.")
else:
    print("❌ NOT READY FOR PHASE 6")
    print("   Major issues need to be fixed first.")

print(f"\nNext Steps:")
if tracker.failed > 0:
    print("1. Review and fix failed tests")
    print("2. Re-run tests to verify fixes")
    print("3. Ensure all core endpoints are working")
else:
    print("1. All tests passed! ✅")
    print("2. Backend is ready for Phase 6 advanced features")
    print("3. Consider adding more edge case tests")

print("\n" + "=" * 60)